<h1>SQLite Database Creator</h1>
<p>This notebook aggreates a range of data collected for the take-a-seat project and converts it into an SQLite database based on the following schema: <img src="schema.png" alt="Schema for Database"></p>
<p>The data is converted into csvs, then added into the SQLite .db</p>

In [142]:
import pandas as pd
import numpy as np
import os
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import *
import re
from matplotlib import pyplot as plt
import sqlite3
import os

<h3>SQLite Handling functions</h3>
<p>Use Pandas to import dataframe into SQLlite</p>

In [143]:
DB = "hockey_data_goalies.db"

#SQL Handle functions
def run_query(q):
    with sqlite3.connect(DB) as conn:
        x = pd.read_sql(q,conn)
    return x
   
#send command
def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.isolation_level = None
        return conn.execute(c)
    
#show tables
def show_tables():
    r = """
    Select name,type
    FROM sqlite_master
    WHERE type in ("table","view");
        """
    return run_query(r)

<h2>Import and Clean up csvs</h2>
    <p><ul>
        <li><b>team_list</b>: This csv gets piped straight into its own table</li>
        <li><b>player_list</b>: Only count those players who have played more than 113 games (median no)</li>
</ul><p>

In [144]:
#import team list data
teams = pd.read_csv('team_list.csv',encoding = "ISO-8859-1")
teams['longitude'] = teams['longitude'].astype(float)
teams.drop(columns=['index'],inplace=True)
teams = teams.set_index('team_abbr')
teams.head()

,team_name,years_active,city,latitude,longitude,timezone
team_abbr,,,,,,
ANA,Anaheim Ducks,2006-2020,Anaheim,33.808,-117.876,-8
MDA,Mighty Ducks of Anaheim,1993-2006,Anaheim,33.808,-117.876,-8
ARI,Arizona Coyotes,2014-2020,Glendale,33.532,-112.261,-7
PHX,Phoenix Coyotes,1996-2014,Glendale,33.532,-112.261,-7
WIN,Winnipeg Jets,1990-1996,Winnipeg,49.893,-97.144,-6


<h2>Wipe Database first</h2>
<p>Since we're building the db from scratch here</p>

In [212]:
tables = show_tables() #gets the tables in the db
with sqlite3.connect(DB) as conn:
    for row,data in tables.iterrows():
        run_command("DROP TABLE IF EXISTS {}".format(tables.loc[row,'name'])) #clear DB of existing tables

<h3>Enter Team Data</h3>

In [213]:
#make a new table with the appropriate keys
c1 = """
    CREATE TABLE IF NOT EXISTS team_list (
    team_abbr TEXT PRIMARY KEY,
    team_name TEXT,
    years_active TEXT,
    city TEXT,
    latitude REAL,
    longitude REAL,
    timezone INTEGER
);
"""
run_command(c1)

#inject data into the table
teams.to_sql('team_list',con=sqlite3.connect(DB), if_exists='append')

<h2>Enter Player Data</h2>

In [214]:
#import player list data
players = pd.read_csv('nhlplayerlistr3.txt','\t')
players.drop(columns=['Unnamed: 0','link','height','weight'],inplace=True)
players = players.set_index('unique_id')
#sort based ons whether or not there is injury data
players = players[(players['position']=='G')&(players['total_games']>10)].copy()

In [215]:
#make a new table with the appropriate keys #drop position as we're only dealing with Goalies
c1 = """
    CREATE TABLE IF NOT EXISTS player_list (
    unique_id TEXT PRIMARY KEY,
    player TEXT,
    year_start INTEGER,
    year_finish INTEGER,
    height_cm INTEGER,
    weight_kg INTEGER,
    total_games INTEGER
);
"""

run_command(c1)

#inject data into the table
players[['player','year_start','year_finish','height_cm','weight_kg','total_games']].to_sql('player_list',con=sqlite3.connect(DB), if_exists='append')

<h2>Enter Game Logs by Team</h2>

In [216]:
#make a new table with the appropriate keys #drop position as we're only dealing with Goalies
c1 = """
    CREATE TABLE IF NOT EXISTS team_log (
    season_id TEXT PRIMARY KEY,
    team_id TEXT,
    date_game TEXT,
    game_location BLOB,
    opp_name TEXT,
    goals INTEGER,
    opp_goals INTEGER,
    game_outcome TEXT,
    overtimes TEXT,
    shots INTEGER,
    shots_against INTEGER,
    FOREIGN KEY (team_id) REFERENCES team_list(team_id)
);
"""

run_command(c1)

In [217]:
#import player list into a dataframe
for files in os.listdir('C:\\Users\\jesse\\Documents\\Projects\\takeaseat\\Data\\team_gamelogs\\'):
    season_log = pd.read_csv('team_gamelogs\\'+files,'\t')
    season_log['team_id'] = files[:3]
    season_log['season_id'] = season_log['team_id']+season_log['date_game'].astype('datetime64').map(lambda x:x.strftime('%Y%m%d'))
    season_log.drop(columns=['Unnamed: 0'],inplace=True)
    season_log = season_log.set_index('season_id')
    season_log[['team_id','date_game','game_location', 'opp_name', 'goals','opp_goals','game_outcome','overtimes','shots','shots_against']].to_sql('team_log',con=sqlite3.connect(DB), if_exists='append')

<h2>Player Logs</h2>
Now add goalie player logs with the analysed injury reports

In [218]:
def injury_match(pdf,inj_dat):
    """
        INPUTS: pdf - the player log to analyse
                inj_data - the injury log to analyse for that player
        OUTPUTS: the injury dataframe to analyse
    """
    for i,i_day in inj_dat.iterrows():
        injury_reported = i_day['Date']

        #find minimum number of dates between game and injury report
        dateoffset = pdf['date_game'].map(lambda x:np.abs(x-injury_reported))
        #closest entry in the player logs
        closest_entry = pdf[dateoffset==dateoffset.min()].iloc[[0]]

        #if the injury date happened before the closest game... 
        if closest_entry['date_game'].values>injury_reported:
            closest_entry = pdf.loc[closest_entry['date_game'].index-1]

        #game before the injury game
        pregame = pdf.loc[closest_entry['date_game'].index-1]
        #make sure it makes sense aka did it happen recently...
        if (((pregame['date_game'].map(lambda x:np.abs(x-injury_reported).days))<15).values):
            pdf.loc[pregame.index,'injured'] = 1   
            #print('Last Game: %s, Injury Report %s'%(pregame['date_game'].values,i_day['Date']))
    return pdf

In [219]:
#make a new table with the appropriate keys
c1 = """
    CREATE TABLE IF NOT EXISTS player_log(
    gamelog_id TEXT PRIMARY KEY,
    player_id TEXT,
    date_game TEXT,
    game_season TEXT,
    age REAL,
    team_id TEXT,
    game_location TEXT,
    opp_id TEXT,
    game_result TEXT,
    goals_against INT,
    shots_against INT,
    saves INT,
    save_pct INT,
    shutouts INT,
    pen_min INT,
    time_on_ice TEXT,
    injured INT,
    FOREIGN KEY (player_id) REFERENCES player_list(unique_id),
    FOREIGN KEY (team_id) REFERENCES team_list(team_id),
    FOREIGN KEY (opp_id) REFERENCES team_list(team_id)
);"""

run_command(c1)
show_tables()

,name,type
0,team_list,table
1,player_list,table
2,team_log,table
3,player_log,table


In [225]:
#for each player in the database
players = run_query('Select * FROM player_list')

#the player list is already filtered for goalies...
for i,player in players.iterrows():
    
    #load player log
    plog = pd.read_csv('..\\Data\\player_gamelogs\\'+player['unique_id']+'.txt')
    plog.drop(columns=['Unnamed: 0'],inplace=True)
    plog['player_id'] = player['unique_id']
    plog['date_game'] = plog['date_game'].astype('datetime64') #convert to datetime
    plog['age'] = (plog['age'].str.extract(r'(\d{2})-').astype(int)+plog['age'].str.extract(r'-(\d{3})').astype(int)/365).round(3)
    plog['injured'] = 0
    #load the injury report
    try:
        inj= pd.read_csv('..\\Data\\player_injurylist\\'+player['unique_id']+'.txt','\t')
        try:
            inj.drop(columns=['Unnamed: 0'],inplace=True)
            inj.columns = inj.columns.str.strip()
            inj['Date'] = inj['Date'].astype('datetime64') #convert to datetime
            injured = inj[inj['Relinquished']!=' '] #all the times put onto the injured list
        except  AssertionError as error:
            print(error)
        
        #now note injuries
        player_logs = injury_match(plog,injured)
        print(player['unique_id'],inj.shape[0])
    except:
        print('Error: %s'% player['unique_id'])
        player_logs = plog.copy()
        #no reported injury
        pass
    
    #create unique gamelog id
    player_logs['gamelog_id'] = player_logs['player_id']+player_logs['date_game'].map(lambda x:x.strftime('%Y%m%d'))
    player_logs['date_game'] = player_logs['date_game'].astype(str)
    player_logs = player_logs.set_index('gamelog_id')
    #inject into SQL database
    player_logs[['player_id','date_game','game_season','age','team_id','game_location','opp_id',
                 'game_result','goals_against','shots_against','saves','save_pct','shutouts',
                'pen_min','time_on_ice','injured']].to_sql('player_log',con=sqlite3.connect(DB), if_exists='append')

Error: aebisda01
allenja01 4
anderfr01 27
andercr01 37
aubinje01 6
auldal01 3
bacasja01 5
bachmri01 2
backsni01 35
Error: bailesc01
Error: balesmi01
barrato01 13
beaupdo01 5
Error: beaurst01
belfoed01 8
Error: bergeje01
bernijo01 26
berrare01 5
Error: berthda01
Error: berubje02
Error: besteal01
bierkza01 2
billicr01 5
Error: binnijo01
bironma01 5
bishobe01 45
blackda01 3
blackma01 5
Error: bluejo01
bobrose01 30
bouchbr01 21
Error: brathfr01
brodema01 22
brossla01 1
Error: brustba01
bryzgil01 15
budajpe01 4
burkese01 14
Error: campbja01
Error: careyji01
caronse01 3
Error: caseyjo01
Error: cassifr01
cechmro01 4
Error: chabofr01
Error: charpse01
cheveti01 1
clemmsc01 8
cloutda01 12
Error: cloutja01
dafoeby01 4
danisya01 1
darlisc01 4
Error: delguma01
Error: deliaco01
Error: dellaa01
Error: demkoth01
denisma01 9
Error: drouije01
Error: desmica01
Error: desropa01
dipieri01 46
Error: divisre01
dominlo01 9
Error: drapeto01
driedch01 3
Error: dubiewa01
dubnyde01 16
dunhami01 9
elliobr01 19
ell

In [229]:
run_query("""
WITH inj_list AS(SELECT player_id,COUNT(gamelog_id) numGames,SUM(injured) inCount
FROM player_log
GROUP BY player_id)
SELECT player_id,numGames,inCount
FROM inj_list
WHERE inCount>0""")

,player_id,numGames,inCount
0,allenja01,289,2
1,andercr01,648,17
2,anderfr01,369,12
3,auldal01,237,1
4,bacasja01,38,1
...,...,...,...
163,vokouto01,700,13
164,wakalda01,185,2
165,wardca01,701,14
166,weekeke01,348,3
